In [ ]:
Итеративные и рекурсивные алгоритмы
Краснов Д.О.

In [ ]:
Задание 1(Рекурсивная реализация)

In [ ]:
import math
import time

def sqrt_recursive(N, A, E):
    if N < 0:
        raise ValueError("N должно быть неотрицательным")
    if N == 0:
        return 0.0
    if A <= 0:
        raise ValueError("A должно быть положительным")
    
    """Новое приближение по формуле Ньютона"""
    new_A = (A + N / A) / 2
    
    if abs(new_A - A) < E:
        return new_A
    
    return sqrt_recursive(N, new_A, E)

print("Введите число для извлечения квадратного корня:")
N = float(input())
print("Введите начальное приближение:")
A = float(input())
print("Введите точность вычисления:")
E = float(input())

try:
    result = sqrt_recursive(N, A, E)
    print(f"Квадратный корень из {N} с точностью {E}:")
    print(f"Результат: {result}")
    print(f"Проверка: {result} * {result} = {result * result}")
except ValueError as e:
    print(f"Ошибка: {e}")

In [ ]:
Задание 2 (Итеративная реализация (без рекурсии))

In [ ]:
def sqrt_iterative(N, A, E):
    if N < 0:
        raise ValueError("N должно быть неотрицательным")
    if N == 0:
        return 0.0
    if A <= 0:
        raise ValueError("A должно быть положительным")
    
    current_A = A
    while True:
        new_A = (current_A + N / current_A) / 2
        if abs(new_A - current_A) < E:
            return new_A
        current_A = new_A

print("Введите число для извлечения квадратного корня:")
N = float(input())
print("Введите начальное приближение:")
A = float(input())
print("Введите точность вычисления:")
E = float(input())

try:
    result = sqrt_iterative(N, A, E)
    print(f"Квадратный корень из {N} с точностью {E}:")
    print(f"Результат: {result}")
    print(f"Проверка: {result} * {result} = {result * result}")
except ValueError as e:
    print(f"Ошибка: {e}")

In [ ]:
Работа декоратора(синтаксис)

In [ ]:
import sys
import time
from functools import wraps

class SqrtStats:
    """Контейнер для статистики выполнения алгоритмов"""
    def __init__(self):
        self.iterations = 0
        self.max_stack_depth = 0
        self.execution_time = 0
        self.intermediate_results = []
    
    def reset(self):
        """Сброс статистики перед новым запуском"""
        self.iterations = 0
        self.max_stack_depth = 0
        self.execution_time = 0
        self.intermediate_results = []

stats = SqrtStats() # Единый объект для всех алгоритмов

def save_intermediate_results(func):
    """Декоратор для автоматического сбора статистики рекурсии"""
    @wraps(func)
    def wrapper(N, A, E, depth=0):
        stats.iterations += 1
        stats.max_stack_depth = max(stats.max_stack_depth, depth)
        
        intermediate = {
            'depth': depth, 'N': N, 'A': A, 'E': E, 'timestamp': time.time()
        }
        
        result = func(N, A, E, depth)
        
        intermediate['result'] = result
        stats.intermediate_results.append(intermediate)
        
        return result
    return wrapper

def manual_sqrt_recursive_with_storage(N, A, E):
    """Рекурсивный алгоритм с ручным сбором статистики"""
    intermediate_results = []
    stack_depth = 0
    
    def recursive_helper(n, a, e, depth):
        nonlocal stack_depth
        stack_depth = max(stack_depth, depth)
        stats.iterations += 1
        
        # Сохраняем состояние до вычислений
        intermediate_results.append({
            'depth': depth, 'N': n, 'A': a, 'E': e, 'action': 'start'
        })
        
        if n < 0:
            raise ValueError("N должно быть неотрицательным")
        if n == 0:
            return 0.0
        if a <= 0:
            raise ValueError("A должно быть положительным")
        
        new_A = (a + n / a) / 2
        
        # Сохраняем результаты вычислений
        intermediate_results.append({
            'depth': depth, 'current_A': a, 'new_A': new_A, 
            'error': abs(new_A - a), 'action': 'calculated'
        })
        
        if abs(new_A - a) < e:
            intermediate_results.append({
                'depth': depth, 'result': new_A, 'action': 'completed'
            })
            return new_A
        
        result = recursive_helper(n, new_A, e, depth + 1)
        return result
    
    result = recursive_helper(N, A, E, 0)
    stats.max_stack_depth = stack_depth
    stats.intermediate_results = intermediate_results
    return result

@save_intermediate_results
def sqrt_recursive_decorated(N, A, E, depth=0):
    """Рекурсивный алгоритм с автоматическим сбором статистики через декоратор"""
    if N < 0:
        raise ValueError("N должно быть неотрицательным")
    if N == 0:
        return 0.0
    if A <= 0:
        raise ValueError("A должно быть положительным")
    
    new_A = (A + N / A) / 2
    
    if abs(new_A - A) < E:
        return new_A
    
    return sqrt_recursive_decorated(N, new_A, E, depth + 1)

def sqrt_iterative_with_stats(N, A, E):
    """Итеративный алгоритм со сбором статистики"""
    stats.reset()
    start_time = time.time()
    
    if N < 0:
        raise ValueError("N должно быть неотрицательным")
    if N == 0:
        stats.execution_time = time.time() - start_time
        return 0.0
    if A <= 0:
        raise ValueError("A должно быть положительным")
    
    current_A = A
    iteration = 0
    
    while True:
        iteration += 1
        stats.iterations = iteration
        
        new_A = (current_A + N / current_A) / 2
        
        stats.intermediate_results.append({
            'iteration': iteration, 'current_A': current_A, 
            'new_A': new_A, 'error': abs(new_A - current_A)
        })
        
        if abs(new_A - current_A) < E:
            break
            
        current_A = new_A
    
    stats.execution_time = time.time() - start_time
    return new_A

def analyze_sqrt_stack_limit():
    """Определение максимальной безопасной глубины рекурсии"""
    original_limit = sys.getrecursionlimit() # Получаем системный лимит (обычно 1000)
    max_safe_iterations = 0
    
    for i in range(original_limit - 100):
        try:
            sqrt_recursive_decorated(2, 1, 1e-300)
            max_safe_iterations = i
        except RecursionError:
            break
    
    return max_safe_iterations, original_limit

def sqrt_performance_comparison(test_cases):
    """Сравнение производительности всех алгоритмов"""
    results = []
    
    for N, A, E in test_cases:
        print(f"\nТестирование sqrt({N}) с A={A}, E={E}:")
        
        # Рекурсивный с декоратором
        stats.reset()
        start_time = time.time()
        try:
            result1 = sqrt_recursive_decorated(N, A, E)
            recursive_time = time.time() - start_time
            print(f"Рекурсивный (декоратор): {result1:.6f}")
            print(f"  Время: {recursive_time:.6f}с, Итерации: {stats.iterations}, Глубина: {stats.max_stack_depth}")
        except Exception as e:
            print(f"Рекурсивный (декоратор): Ошибка - {e}")
            recursive_time = float('inf')
        
        # Рекурсивный ручной
        stats.reset()
        start_time = time.time()
        try:
            result2 = manual_sqrt_recursive_with_storage(N, A, E)
            manual_time = time.time() - start_time
            print(f"Рекурсивный (ручной): {result2:.6f}")
            print(f"  Время: {manual_time:.6f}с, Итерации: {stats.iterations}, Глубина: {stats.max_stack_depth}")
        except Exception as e:
            print(f"Рекурсивный (ручной): Ошибка - {e}")
            manual_time = float('inf')
        
        # Итеративный
        stats.reset()
        try:
            result3 = sqrt_iterative_with_stats(N, A, E)
            iterative_time = stats.execution_time
            print(f"Итеративный: {result3:.6f}")
            print(f"  Время: {iterative_time:.6f}с, Итерации: {stats.iterations}")
        except Exception as e:
            print(f"Итеративный: Ошибка - {e}")
            iterative_time = float('inf')
        
        # Проверка совпадения результатов
        try:
            if abs(result1 - result2) < E and abs(result2 - result3) < E:
                print("  Все алгоритмы дали одинаковый результат")
            else:
                print("  Результаты различаются")
        except:
            print("  Невозможно сравнить результаты")
        
        results.append({
            'N': N, 'A': A, 'E': E,
            'recursive_time': recursive_time,
            'manual_time': manual_time,
            'iterative_time': iterative_time,
            'iterations': stats.iterations,
            'stack_depth': stats.max_stack_depth
        })
    
    return results

def print_intermediate_results():
    """Вывод промежуточных результатов"""
    print("\nПромежуточные результаты:")
    for i, result in enumerate(stats.intermediate_results[-5:]):
        print(f"  {i+1}: {result}")

if __name__ == "__main__":
    print("=" * 50)
    print("АНАЛИЗ АЛГОРИТМОВ ВЫЧИСЛЕНИЯ КВАДРАТНОГО КОРНЯ")
    print("=" * 50)
    
   
    print("\n1. АНАЛИЗ ОГРАНИЧЕНИЙ РЕКУРСИИ")
    max_iterations, limit = analyze_sqrt_stack_limit()
    print(f"Лимит рекурсии: {limit}")
    print(f"Безопасное количество итераций: {max_iterations}")
    
  # Тестовые данные
    test_cases = [
        (25, 5, 1e-10),
        (2, 1, 1e-10),
        (100, 10, 1e-12),
        (1000, 30, 1e-8),
        (0.25, 0.5, 1e-10),
    ]
    
    print("\n2. СРАВНЕНИЕ ПРОИЗВОДИТЕЛЬНОСТИ")
    results = sqrt_performance_comparison(test_cases)
    
    
    print("\n3. СВОДНАЯ СТАТИСТИКА")
    print("Алгоритм           | Время     | Итерации | Глубина")
    print("-" * 50)
    
    successful = [r for r in results if r['recursive_time'] != float('inf')]
    
    if successful:
        recursive_times = [r['recursive_time'] for r in successful]
        manual_times = [r['manual_time'] for r in successful]
        iterative_times = [r['iterative_time'] for r in successful]
        
        print(f"Рекурсивный (декор) | {sum(recursive_times)/len(recursive_times):.6f}с | "
              f"{max(r['iterations'] for r in successful):<8} | {max(r['stack_depth'] for r in successful)}")
        
        print(f"Рекурсивный (ручной) | {sum(manual_times)/len(manual_times):.6f}с | "
              f"{max(r['iterations'] for r in successful):<8} | {max(r['stack_depth'] for r in successful)}")
        
        print(f"Итеративный         | {sum(iterative_times)/len(iterative_times):.6f}с | "
              f"{max(r['iterations'] for r in successful):<8} | -")
    
    
    print("\n4. ДЕМОНСТРАЦИЯ ")
    stats.reset()
    try:
        sqrt_recursive_decorated(25, 5, 1e-10)
        print_intermediate_results()
    except Exception as e:
        print(f"Ошибка: {e}")
    
    print("\n" + "=" * 50)